In [20]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
from torch import optim

In [21]:
torch.cuda.is_available()
 

True

In [22]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


# Dataset Class

In this section, we will use the previous code to build a dataset class. As before, make sure the even sampes are positve and the odd samples are negative. If parameter `train` is set to **True** , use the first 30000 samples as training data; otherwise, the remaining samples will be used as validation data. Do not forget to sort you files so they are in the same order.

In [7]:
class Dataset(Dataset):
    
    def __init__(self, transform=None , train=True):
        
        directory="./resources/data"
        positive="Positive"
        negative="Negative"
        
        positive_file_path=os.path.join(directory , positive)
        negative_file_path=os.path.join(directory , negative)
        
        positive_files=[os.path.join(positive_file_path , file) for file in os.listdir(positive_file_path) if file.endswith(".jpg")]
        positive_files.sort()
        
        negative_files=[os.path.join(negative_file_path , file) for file in os.listdir(negative_file_path) if file.endswith(".jpg")]
        negative_files.sort()
        
        number_of_samples = len(positive_files) + len(negative_files)
        
        self.all_files = [None] * number_of_samples
        self.all_files[::2] = positive_files
        self.all_files[1::2] = negative_files
        
        self.transform=transform
        
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2] = 1
        self.Y[1::2] = 0
        
        if train:
            self.all_files = self.all_files[:30000]
            self.Y = self.Y[:30000]
            self.len = len(self.all_files)
        else:
            self.all_files = self.all_files[30000:]
            self.Y = self.Y[30000:]
            self.len = len(self.all_files)
    
    def __getitem__ (self, idx):
        
        image=Image.open(self.all_files[idx])
        y = self.Y[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, y
    
    def __len__(self):
        return self.len
    
    # note to self, in pytorch the target has to be a single number from the inteval [0,#classes]
    #instead of being one hot encoded vector
    def vectorize(y, number_of_classes):
        out = torch.zeros(number_of_classes).type(torch.IntTensor)    
        out[y] = 1
        
        return out

## Transform Object and Dataset Object

Create a transform object, that uses the `Compose` function. First use the transform `ToTensor()` and followed by `Normalize(mean, std)`. The value for **mean** and **std** are provided.

In [8]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

Create an object for the training data `dataset_train` and validation `dataset_val`. Use the transform object to convert the images to tensors using the transform object:

In [9]:
dataset_train=Dataset(transform=transform, train=True)
dataset_val=Dataset(transform=transform, train=False)

We can find the shape of the image :

In [10]:
dataset_train[0][0].shape

torch.Size([3, 227, 227])

In [11]:
size_of_image = 3*227*227
size_of_image

154587

Create a custom module for Softmax for two classes, called model. The input size should be the `size_of_image`, you should record the maximum accuracy achieved on the validation data for the different epochs. For example, if after 5 epochs the accuracy was *0.5,0.2,0.64,0.77,0.66* you would select 0.77.

Train the model with the following free parameter values:

**Parameter Values**

* learning rate:0.1
* momentum term:0.1
* batch size training : 1000
* loss function: Cross Entropy Loss
* epochs: 5
* set torch.manual_seed(0)

In [12]:
torch.manual_seed(0)

Custom Module:

In [13]:
class SoftMax(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(SoftMax, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.linear(x)
        x = self.softmax(x)
        return x

Model Object:

In [14]:
model = SoftMax(size_of_image, 2)

Optimizer:

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001 , momentum=0.1)

Criterion:

In [16]:
cost = nn.CrossEntropyLoss()

Data Loader Training and Validation:

In [17]:
train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1000)
validation_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=1000)

Train Model with 5 epochs, should take 35 minutes:

In [18]:
def train_model(epochs, model, optimizer, costFunc , training_loader, validation_loader, num_of_test ):
    
    accuracy_list = []
    
    for epoch in range(epochs):
        for X, y in training_loader:
        
            optimizer.zero_grad()
            
            yhat = model(X.view(-1, 3 * 227 * 227))  
                        
            loss = costFunc(yhat , y)
            loss.backward()
            
            optimizer.step()
        
        correct = 0    
        for X_val , y_val in validation_loader:
   
            yhat_val = model(X_val.view(-1, 3 * 227 * 227))
                        
            _ , prediction = torch.max(yhat_val, 1)
            correct += (prediction == y_val).sum().item()
        
        accuracy = correct / num_of_test
        print(accuracy)
        accuracy_list.append(accuracy)
        
    return accuracy_list

In [19]:
%%time

accuracies = train_model(5, model, optimizer, cost, train_loader, validation_loader, len(dataset_val))

0.7237
0.7297
0.7319
0.7357
0.7391
Wall time: 7min 15s


In [15]:
accuracies

[0.7237, 0.7297, 0.7319, 0.7357, 0.7391]

In [16]:
print(dataset_val[1][1])

tensor(0)


In [248]:
len(dataset_val)

10000

In [353]:
model2 = SoftMax(size_of_image, 2)

In [45]:
s = model(dataset_train[4][0].view(-1, 3* 227*227))

In [46]:
s

tensor([[0.4296, 0.5704]], grad_fn=<SoftmaxBackward>)

In [44]:
dataset_train[4][1]

tensor(1)

In [357]:
t = model2(dataset_train[8][0].view(-1, 3* 227*227))

In [358]:
t

tensor([[0.6491, 0.3509]], grad_fn=<SoftmaxBackward>)

In [367]:
val , predict = torch.max(t.data, 1)

In [368]:
predict

tensor([0])

In [369]:
val

tensor([0.6491])

In [280]:
dataset_train[0][0]

tensor([[[0.1083, 0.1254, 0.1768,  ..., 0.2453, 0.2453, 0.2453],
         [0.1426, 0.1597, 0.1768,  ..., 0.2453, 0.2453, 0.2453],
         [0.2111, 0.1939, 0.1939,  ..., 0.2453, 0.2453, 0.2453],
         ...,
         [0.2282, 0.2453, 0.2453,  ..., 0.2624, 0.2624, 0.2624],
         [0.2282, 0.2453, 0.2453,  ..., 0.2624, 0.2624, 0.2624],
         [0.2282, 0.2453, 0.2453,  ..., 0.2624, 0.2624, 0.2624]],

        [[0.3102, 0.3277, 0.3803,  ..., 0.4153, 0.4153, 0.4153],
         [0.3452, 0.3627, 0.3803,  ..., 0.4153, 0.4153, 0.4153],
         [0.4153, 0.3978, 0.3978,  ..., 0.4153, 0.4153, 0.4153],
         ...,
         [0.3803, 0.3978, 0.3978,  ..., 0.4328, 0.4328, 0.4328],
         [0.3803, 0.3978, 0.3978,  ..., 0.4328, 0.4328, 0.4328],
         [0.3803, 0.3978, 0.3978,  ..., 0.4328, 0.4328, 0.4328]],

        [[0.5485, 0.5659, 0.6182,  ..., 0.6182, 0.6182, 0.6182],
         [0.5834, 0.6008, 0.6182,  ..., 0.6182, 0.6182, 0.6182],
         [0.6531, 0.6356, 0.6356,  ..., 0.6182, 0.6182, 0.

In [287]:
dataset_train[4][1]

tensor(1)

In [268]:
list(model.parameters())[0]

Parameter containing:
tensor([[ 0.0054,  0.0069,  0.0036,  ...,  0.0045,  0.0050,  0.0060],
        [-0.0048, -0.0037, -0.0066,  ..., -0.0071, -0.0071, -0.0070]],
       requires_grad=True)

# Individual Softmax Steps

In [34]:
h = torch.tensor([[1.,3.,4.,5.,6.,7.,8.], [6.,4.,6.,3.,7.,2.,1.], [3.,1.,1.,1.,1.,2.,1.], [3.5,7.,7.,5.,8.,2.,8.,]])

In [35]:
h.shape

torch.Size([4, 7])

In [36]:
linear = nn.Linear(7 , 3)

In [37]:
z = linear(h)
z

tensor([[-1.6249, -3.7367,  2.5930],
        [ 0.1869, -2.9774,  1.7395],
        [-1.2051, -0.7701,  1.1830],
        [-0.2920, -4.3519,  2.0014]], grad_fn=<AddmmBackward>)

In [38]:
softmax = nn.Softmax(dim = 1)

In [39]:
s = softmax(z)
s

tensor([[0.0145, 0.0018, 0.9838],
        [0.1734, 0.0073, 0.8192],
        [0.0744, 0.1150, 0.8106],
        [0.0915, 0.0016, 0.9069]], grad_fn=<SoftmaxBackward>)

In [40]:
s.data

tensor([[0.0145, 0.0018, 0.9838],
        [0.1734, 0.0073, 0.8192],
        [0.0744, 0.1150, 0.8106],
        [0.0915, 0.0016, 0.9069]])

In [41]:
_ , yhat = torch.max(s, 1)

In [42]:
yhat

tensor([2, 2, 2, 2])

In [43]:
yhat.shape

torch.Size([4])

In [44]:
torch.ones([4]).type(torch.IntTensor)

tensor([1, 1, 1, 1], dtype=torch.int32)

In [45]:
def vectorize(y, number_of_classes):
    out = torch.zeros(number_of_classes).type(torch.IntTensor)    
    out[y] = 1
    return out


vectorize(0 , 4)

tensor([1, 0, 0, 0], dtype=torch.int32)